# Introduction

This notebook is for the major project submission for COMP7220/8220, on the language dataset and task. It contains the following sections:

i. Importing the Data packages.

ii. Data Preprocessing.

iii. Conventional Machine Learning Approach.
    
    a. Logistic Regression Model.
    
    b. Linear SVM. 

iv. Deep Learning approach.

    a. Long short-term memory Model.

v. Public dataset.

vi. Private dataset.

# IMPORTING THE DATA PACKAGES

In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from textblob import Word
import re
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import nltk
from nltk.corpus import stopwords
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from os.path import join 
import pickle
import string
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
stemmer= PorterStemmer()
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
from keras.layers import SpatialDropout1D
from keras.layers import LSTM
from keras.callbacks import EarlyStopping
from keras.layers import Dropout

Using TensorFlow backend.


In [2]:
def loading_pickle(path):
    with open(path, 'rb') as f:
        file = pickle.load(f)
        print ('Loaded %s..' %path)
        return file

dataset_path =('C:/Users/sadap/Downloads/ML proj')
emotions = ['anger', 'fear', 'joy', 'sadness']

trainingtweet = np.load(join(dataset_path, 'text_train_tweets.npy'))
labeltraining = np.load(join(dataset_path, 'text_train_labels.npy'))
vocab_list = loading_pickle(join(dataset_path, 'text_word_to_idx.pkl'))

tweetvalue = np.load(join(dataset_path, 'text_val_tweets.npy'))
labelvalue = np.load(join(dataset_path, 'text_val_labels.npy'))

publictweets_test= np.load(join(dataset_path, 'text_test_public_tweets_rand.npy'))

print(len(vocab_list))
idx_to_word = {i: w for w, i in vocab_list.items()}
for i in range(7):
  print(i, idx_to_word[i])

Loaded C:/Users/sadap/Downloads/ML proj\text_word_to_idx.pkl..
13978
0 <NULL>
1 <START>
2 <END>
3 it
4 makes
5 me
6 so


# Data Preprocessing

In [3]:
def stringconversion(text):
    string_list=[]
    for arr in text:
        string = ""
        for i in arr:
            if i>2: 
                string+= idx_to_word[i]
                string+= " "
        string_list.append(string)
    return string_list

trainingtweet=stringconversion(trainingtweet)
tweetvalue=stringconversion(tweetvalue)
trainingtweet=pd.DataFrame(trainingtweet)
tweetvalue=pd.DataFrame(tweetvalue)


labeltraining=pd.DataFrame(labeltraining)
labelvalue=pd.DataFrame(labelvalue)


labeltraining.rename(columns={0:'Emotion'},inplace=True)
labelvalue.rename(columns={0:'Emotion'},inplace=True)

In [4]:
def tweet_preprocessing(TXT):
    TXT = TXT.astype(str)

    TXT[0] = TXT[0].apply(lambda A: A.lower())
    
    TXT[0] = TXT[0].apply(lambda A: word_tokenize(A))

    TXT = TXT.astype(str)
    
    TXT[0] = TXT[0].str.replace('[^\w\s]',' ')
    

    thamba = stopwords.words('english')
    TXT[0] = TXT[0].apply(lambda A: " ".join(A for A in A.split() if A not in thamba))


    TXT[0] = TXT[0].apply(lambda A: lemma_conversion(A))


    TOKEN_T = TXT[0].apply(lambda A: A.split())
    ps = PorterStemmer()
    TOKEN_T = TOKEN_T.apply(lambda A: [ps.stem(i) for i in A])

    for i in range(len(TOKEN_T)):
        TOKEN_T[i] = ' '.join(TOKEN_T[i])
    TXT[0] = TOKEN_T
    return TXT

def lemma_conversion(sentence):
    sent = TextBlob(sentence)
    TL = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    W_T= [(w, TL.get(pos[0], 'n')) for w, pos in sent.tags]    
    LemA_list = [wd.lemmatize(tag) for wd, tag in W_T]
    return " ".join(LemA_list)

We have used Lemmatization to transform a word to it's root form.

In [5]:
trainingtweet=tweet_preprocessing(trainingtweet)
tweetvalue=tweet_preprocessing(tweetvalue)
Init_DF=np.r_[trainingtweet,tweetvalue]
Init_DF=pd.DataFrame(Init_DF)
up_DF=np.r_[labeltraining,labelvalue]
up_DF=pd.DataFrame(up_DF)

# Conventional Machine Learning Approach.

Data PreProcessing.

In [6]:
Encoder = preprocessing.LabelEncoder()
y = Encoder.fit_transform(up_DF.values)
X_train, X_val, y_train, y_val = train_test_split(Init_DF[0].values, y, stratify=y, random_state=43, test_size=0.2, shuffle=True)


C:\Users\sadap\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Extracting the word vector

In [7]:
V_C= CountVectorizer(analyzer='word')
V_C.fit(X_train)
X_vec =  V_C.transform(X_train)
val_vec =  V_C.transform(X_val)

Using Logistic Regression Model and Linear SVM.

In [8]:
#LogisticRegression Model

L_R_Model = LogisticRegression(C=1)
L_R_Model.fit(X_vec, y_train)
y_pred = L_R_Model.predict(val_vec)
LR_ac = (accuracy_score(y_pred, y_val)*100)

#Linear SVM
LSVM_MODEL = SGDClassifier(alpha=0.001, random_state=5, max_iter=15, tol=None)
LSVM_MODEL.fit(X_vec, y_train)
y_pred = LSVM_MODEL.predict(val_vec)
LVSM_AC = (accuracy_score(y_pred, y_val)*100)

print('Logistic Regression accuracy:', LR_ac)

print('LVSM accuracy:', LVSM_AC)

C:\Users\sadap\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\sadap\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Logistic Regression accuracy: 77.33644859813083
LVSM accuracy: 80.19859813084112


ANALYSIS

In the conventional machine learning approach, LVSM model gives us the better accuracy as compared to linear regression model. As we can see that the LVSM Model acts better on the language data set provided on the kaggle. The prediction rate of the LVSM Model is 3% more than that of the other Conventional machine learning approach. 

In [9]:
L =stringconversion(publictweets_test) 
STAR_DF=pd.DataFrame(L)
STAR_DF=tweet_preprocessing(STAR_DF)
STAR_DF.head()

,0
0,omg mother daughter dull ni move dad worri
1,happi birthday user repeat miss excit back flo...
2,ever cri middl bomb rest someon wake emerg sle...
3,mental suffer worthless pain
4,user courag driver shot bu show courag natur s...


All the string has been converted to word, i.e  tweets numbers- tweet words.

In [10]:
COUNTING_TWEETS= V_C.transform(STAR_DF[0])
PREDICTING_TWEETS= LSVM_MODEL.predict(COUNTING_TWEETS)
STAR_DF['Prediction']=pd.DataFrame(PREDICTING_TWEETS)
STAR_DF.set_index(pd.Index(range(1,4065)),inplace=True)
STAR_DF.index.name='ID'
PT_csv=STAR_DF.drop(0,axis=1)
PT_csv.head()

,Prediction
ID,
1,3
2,2
3,1
4,0
5,1


In [11]:
PT_csv.to_csv(r"C:\Users\sadap\Downloads\ML proj\45817901_conventional",index=True)

# Deep Learning Approach

In [12]:
fukat_data=pd.read_csv("tweets_lab.csv")
fukat_data.rename(columns={'Tweet':0},inplace=True)
fukat_tweetX=fukat_data[0]
fukat_data.index.name='ID'
fukat_tweetY=fukat_data['Emotion']
fukat_tweetX=pd.DataFrame(fukat_tweetX)
fukat_tweetY=pd.DataFrame(fukat_tweetY)
fukat_tweetX=tweet_preprocessing(fukat_tweetX)
fukat_tweetY=fukat_tweetY.replace({'anger':0,'fear':1,'joy':2,'sadness':3})

In [13]:
fukat_tweetY.head()

,Emotion
ID,
0,0
1,0
2,0
3,0
4,0


In [14]:
M_DF=np.r_[Init_DF,fukat_tweetX]
M_DF=pd.DataFrame(M_DF)
sent_M_DF=np.r_[up_DF,fukat_tweetY]
sent_M_DF=pd.DataFrame(sent_M_DF)
sent_M_DF.rename(columns={0:'Emotion'},inplace=True)

In [15]:
NEW_WORDS= 5000
SEQ_LEN = 26
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=NEW_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(M_DF[0].values)
word_index = tokenizer.word_index

X = tokenizer.texts_to_sequences(M_DF[0].values)
X = pad_sequences(X, maxlen= SEQ_LEN)
Y = pd.get_dummies(sent_M_DF['Emotion']).values

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(10958, 26) (10958, 4)
(4697, 26) (4697, 4)


Strings are converted to tokens, all the essential filters have been apllied such as the characters such as brackets,comma,etc has been removed and the upper cases word are converted to lower case. 

In [16]:
DL_Model = Sequential()
DL_Model.add(Embedding(NEW_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
DL_Model.add(SpatialDropout1D(0.2))
DL_Model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
DL_Model.add(Dense(4, activation='softmax'))
DL_Model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
epochs = 10
batch_size = 64
history = DL_Model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

C:\Users\sadap\Anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 9862 samples, validate on 1096 samples
Epoch 1/10
9862/9862 [==============================] - 8s 846us/step - loss: 1.0644 - accuracy: 0.5772 - val_loss: 0.5626 - val_accuracy: 0.8221
Epoch 2/10
9862/9862 [==============================] - 8s 769us/step - loss: 0.3969 - accuracy: 0.8745 - val_loss: 0.4445 - val_accuracy: 0.8513
Epoch 3/10
9862/9862 [==============================] - 7s 759us/step - loss: 0.2784 - accuracy: 0.9086 - val_loss: 0.4228 - val_accuracy: 0.8659
Epoch 4/10
9862/9862 [==============================] - 7s 746us/step - loss: 0.2413 - accuracy: 0.9197 - val_loss: 0.4391 - val_accuracy: 0.8604
Epoch 5/10
9862/9862 [==============================] - 7s 712us/step - loss: 0.2076 - accuracy: 0.9262 - val_loss: 0.4168 - val_accuracy: 0.8723
Epoch 6/10
9862/9862 [==============================] - 7s 729us/step - loss: 0.1908 - accuracy: 0.9280 - val_loss: 0.4319 - val_accuracy: 0.8686
Epoch 7/10
9862/9862 [==============================] - 7s 722us/step - loss

Long Short Term Memory model is used which is a most efficient model in Deep Learning Approach, in this for string conversion "ADAM" optimizer is used, "SOFTMAX" activation function is used for multiple classes,here INPUT IS 100 and the output will be in the emotions category mentioned above.

In [17]:
DL_Acc = DL_Model.evaluate(X_test,Y_test)
print('LSTM Accuracy :',DL_Acc[1])

4697/4697 [==============================] - 1s 213us/step
LSTM Accuracy : 0.8592718839645386


# Public Data Set

In [18]:
STAR_DF=pd.DataFrame(stringconversion(publictweets_test))
STAR_DF=tweet_preprocessing(STAR_DF)
STAR_DF.head()

,0
0,omg mother daughter dull ni move dad worri
1,happi birthday user repeat miss excit back flo...
2,ever cri middl bomb rest someon wake emerg sle...
3,mental suffer worthless pain
4,user courag driver shot bu show courag natur s...


In [19]:
FL=[]
for i in range(0,4064):
    sequence = tokenizer.texts_to_sequences(STAR_DF.iloc[i])
    padded = pad_sequences(sequence, maxlen=SEQ_LEN)
    pred = DL_Model.predict(padded)
    FL.append(np.argmax(pred))

In [20]:
FL_DF=pd.DataFrame(FL)
FL_DF.rename(columns={0:'Prediction'},inplace=True)
FL_DF.set_index(pd.Index(range(1,4065)),inplace=True)
FL_DF.index.name='ID'
FL_DF.head()

,Prediction
ID,
1,1
2,2
3,1
4,3
5,1


In [21]:
FL_DF.to_csv(r"C:\Users\sadap\Downloads\ML proj\45817901_Deeplearning",index=True)

# PRIVATE TEST SET

In [22]:
PVT = np.load(join(dataset_path, 'text_test_private_tweets.npy'))

In [23]:
PVT_test=pd.DataFrame(stringconversion(PVT))
PVT_test=tweet_preprocessing(PVT_test)

In [24]:
AKHRI_LIST=[]
for i in range(len(PVT_test)):
    t = tokenizer.texts_to_sequences(PVT_test.iloc[i])
    v = pad_sequences(t, maxlen=SEQ_LEN)
    pred = DL_Model.predict(v)
    AKHRI_LIST.append(np.argmax(pred))

In [25]:
FL_DF_NEW=pd.DataFrame(AKHRI_LIST)

In [26]:
FL_DF_NEW.rename(columns={0:'Prediction'},inplace=True)
FL_DF_NEW.set_index(pd.Index(range(len(PVT_test))),inplace=True)
FL_DF_NEW.index.name='ID'

In [27]:
FL_DF_NEW.head()

,Prediction
ID,
0,2
1,2
2,1
3,1
4,1


In [28]:
FL_DF.to_csv(r"C:\Users\sadap\Downloads\ML proj\45817901_Deeplearning",index=True)

# Conclusion.


In the both approaches, performance is the key. The Deep Learning Approach does not work efficiently when there is samll data, but here the data is massive and the Conventional Machine Learning Approach is not efficient because the accuracy is less compared to the final LSTM model we have used.

# References


1.https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17


2.https://towardsdatascience.com/introduction-to-data-preprocessing-in-machine-learning-a9fa83a5dc9d


3.https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html


4.https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/